# Forward-Forward MNIST Classification

Extending Forward-Forward to 10-class MNIST with minimal hidden neurons (12).

## Architecture

```
Input: 784 (28×28 flattened) + 10 (one-hot label) = 794
Hidden: 12 SingleDendrite neurons
Output: Goodness (sum of squared activations)
```

## Inference (10 forward passes)

```
For each digit d ∈ {0,1,...,9}:
    X_embedded = [image_pixels, one_hot(d)]
    goodness_d = forward(X_embedded)
Predict: argmax(goodness_0, ..., goodness_9)
```

## Hardware Compatibility

- Goodness = mean(I²) = power measurement
- Label embedding = optical input modulation
- No backward pass needed for inference

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import gzip
import urllib.request

from soen_toolkit.core import (
    ConnectionConfig,
    LayerConfig,
    SimulationConfig,
    SOENModelCore,
)

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")

## 1. Load MNIST Dataset

In [ ]:
# Direct MNIST download without torchvision
def download_mnist(data_dir='./data/mnist'):
    """Download MNIST dataset without torchvision."""
    os.makedirs(data_dir, exist_ok=True)
    
    base_url = 'https://ossci-datasets.s3.amazonaws.com/mnist/'
    files = {
        'train_images': 'train-images-idx3-ubyte.gz',
        'train_labels': 'train-labels-idx1-ubyte.gz',
        'test_images': 't10k-images-idx3-ubyte.gz',
        'test_labels': 't10k-labels-idx1-ubyte.gz',
    }
    
    paths = {}
    for key, filename in files.items():
        filepath = os.path.join(data_dir, filename)
        if not os.path.exists(filepath):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(base_url + filename, filepath)
        paths[key] = filepath
    
    return paths


def load_mnist_images(filepath):
    """Load MNIST images from gzipped IDX file."""
    with gzip.open(filepath, 'rb') as f:
        # Read header
        magic = int.from_bytes(f.read(4), 'big')
        n_images = int.from_bytes(f.read(4), 'big')
        n_rows = int.from_bytes(f.read(4), 'big')
        n_cols = int.from_bytes(f.read(4), 'big')
        # Read data
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(n_images, n_rows * n_cols).astype(np.float32) / 255.0


def load_mnist_labels(filepath):
    """Load MNIST labels from gzipped IDX file."""
    with gzip.open(filepath, 'rb') as f:
        # Read header
        magic = int.from_bytes(f.read(4), 'big')
        n_labels = int.from_bytes(f.read(4), 'big')
        # Read data
        return np.frombuffer(f.read(), dtype=np.uint8)


# Download and load MNIST
paths = download_mnist()
X_train_full = torch.from_numpy(load_mnist_images(paths['train_images']))
y_train_full = torch.from_numpy(load_mnist_labels(paths['train_labels'])).long()
X_test_full = torch.from_numpy(load_mnist_images(paths['test_images']))
y_test_full = torch.from_numpy(load_mnist_labels(paths['test_labels'])).long()

print(f"Full dataset: Train={X_train_full.shape}, Test={X_test_full.shape}")

# Use subset for faster experimentation
N_TRAIN = 5000
N_TEST = 1000

torch.manual_seed(42)
train_idx = torch.randperm(len(X_train_full))[:N_TRAIN]
test_idx = torch.randperm(len(X_test_full))[:N_TEST]

X_train = X_train_full[train_idx]
y_train = y_train_full[train_idx]
X_test = X_test_full[test_idx]
y_test = y_test_full[test_idx]

# Scale to SOEN operating range [0.025, 0.275]
X_train = X_train * 0.25 + 0.025
X_test = X_test * 0.25 + 0.025

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")
print(f"X range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"Class distribution (train): {torch.bincount(y_train)}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    img = X_train[i].view(28, 28).numpy()
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Label: {y_train[i].item()}')
    ax.axis('off')
plt.suptitle('MNIST Samples (scaled to SOEN range)')
plt.tight_layout()
plt.show()

## 2. Forward-Forward Functions

In [ ]:
N_CLASSES = 10
SEQ_LEN = 50
LABEL_SCALE = 0.25  # Strong label signal

def embed_label(X, y, n_classes=N_CLASSES, label_scale=LABEL_SCALE):
    """
    Embed one-hot label into input.
    
    Args:
        X: [N, 784] flattened MNIST images
        y: [N] class labels (0-9)
    
    Returns:
        X_embedded: [N, 794] image + one-hot label
    """
    N = X.shape[0]
    one_hot = torch.zeros(N, n_classes)
    one_hot.scatter_(1, y.unsqueeze(1), label_scale)
    return torch.cat([X, one_hot], dim=1)


def create_positive_negative_pairs(X, y, n_classes=N_CLASSES, label_scale=LABEL_SCALE):
    """
    Create positive and negative samples for Forward-Forward.
    
    Positive: image with correct label
    Negative: image with random wrong label
    """
    N = X.shape[0]
    
    # Positive: correct labels
    X_pos = embed_label(X, y, n_classes, label_scale)
    
    # Negative: random wrong labels
    y_wrong = (y + torch.randint(1, n_classes, (N,))) % n_classes
    X_neg = embed_label(X, y_wrong, n_classes, label_scale)
    
    return X_pos, X_neg


# Test embedding
X_pos, X_neg = create_positive_negative_pairs(X_train[:5], y_train[:5])
print(f"Embedded shape: {X_pos.shape}")
print(f"Input dim: 784 pixels + 10 label = 794")

In [ ]:
def compute_goodness(activations):
    """
    Compute goodness as mean of squared activations.
    Hardware-compatible: measures mean power in the layer.
    """
    return (activations ** 2).mean(dim=1)


def forward_forward_loss(goodness_pos, goodness_neg, threshold=0.1, margin=0.05):
    """
    Forward-Forward loss with contrastive term.
    """
    loss_pos = F.softplus(threshold - goodness_pos).mean()
    loss_neg = F.softplus(goodness_neg - threshold).mean()
    contrastive = F.softplus(margin - (goodness_pos - goodness_neg)).mean()
    return loss_pos + loss_neg + contrastive

## 3. Build SOEN Model for MNIST

In [ ]:
def build_ff_mnist_model(hidden_dims, input_dim=794, dt=50.0):
    """
    Build a SOEN model for Forward-Forward MNIST.
    
    Args:
        hidden_dims: List of hidden layer dimensions (e.g., [12] or [12, 12])
        input_dim: 784 pixels + 10 label = 794
    """
    sim_cfg = SimulationConfig(
        dt=dt,
        input_type="state",
        track_phi=False,
        track_power=False,
    )
    
    layers = []
    connections = []
    
    # Input layer
    layers.append(LayerConfig(
        layer_id=0,
        layer_type="Input",
        params={"dim": input_dim},
    ))
    
    # Hidden layers
    for i, hidden_dim in enumerate(hidden_dims):
        layer_id = i + 1
        
        layers.append(LayerConfig(
            layer_id=layer_id,
            layer_type="SingleDendrite",
            params={
                "dim": hidden_dim,
                "solver": "FE",
                "source_func": "Heaviside_fit_state_dep",
                "phi_offset": 0.02,
                "bias_current": 1.98,
                "gamma_plus": 0.0005,
                "gamma_minus": 1e-6,
                "learnable_params": {
                    "phi_offset": False,
                    "bias_current": False,
                    "gamma_plus": False,
                    "gamma_minus": False,
                },
            },
        ))
        
        connections.append(ConnectionConfig(
            from_layer=layer_id - 1,
            to_layer=layer_id,
            connection_type="all_to_all",
            learnable=True,
            params={"init": "xavier_uniform"},
        ))
    
    model = SOENModelCore(
        sim_config=sim_cfg,
        layers_config=layers,
        connections_config=connections,
    )
    
    return model


# Test model
HIDDEN_DIMS = [12]  # Only 12 hidden neurons!
test_model = build_ff_mnist_model(HIDDEN_DIMS)
print(f"Model architecture: 794 → {HIDDEN_DIMS} → goodness")
print(f"Parameters: {sum(p.numel() for p in test_model.parameters() if p.requires_grad)}")
print(f"  (794 × 12 = 9528 weights for first layer)")

## 4. Training Functions

In [ ]:
def evaluate_ff_mnist_fast(model, X, y, batch_size=200, seq_len=10):
    """
    Fast evaluation by batching all 10 class hypotheses together.
    
    Instead of 10 separate forward passes per batch, we concatenate all
    hypotheses into a single large batch for one forward pass.
    """
    model.eval()
    N = X.shape[0]
    all_predictions = []
    
    with torch.no_grad():
        for start in range(0, N, batch_size):
            end = min(start + batch_size, N)
            X_batch = X[start:end]  # [B, 784]
            B = X_batch.shape[0]
            
            # Repeat each sample 10 times (one for each class hypothesis)
            X_repeated = X_batch.unsqueeze(1).expand(-1, N_CLASSES, -1)  # [B, 10, 784]
            X_repeated = X_repeated.reshape(B * N_CLASSES, 784)  # [B*10, 784]
            
            # Create labels 0-9 for each sample
            y_hypotheses = torch.arange(N_CLASSES).unsqueeze(0).expand(B, -1)  # [B, 10]
            y_hypotheses = y_hypotheses.reshape(B * N_CLASSES)  # [B*10]
            
            # Embed all hypotheses
            X_embedded = embed_label(X_repeated, y_hypotheses)  # [B*10, 794]
            X_seq = X_embedded.unsqueeze(1).expand(-1, seq_len, -1)  # [B*10, seq_len, 794]
            
            # Single forward pass for all hypotheses
            _, layer_states = model(X_seq)
            
            # Compute goodness for all
            total_goodness = torch.zeros(B * N_CLASSES)
            for layer_idx in range(1, len(model.layers)):
                act = layer_states[layer_idx][:, -1, :]  # [B*10, hidden]
                total_goodness += compute_goodness(act)
            
            # Reshape to [B, 10] and get predictions
            goodness_matrix = total_goodness.reshape(B, N_CLASSES)
            predictions = goodness_matrix.argmax(dim=1)
            all_predictions.append(predictions)
    
    all_predictions = torch.cat(all_predictions)
    accuracy = (all_predictions == y).float().mean().item()
    model.train()
    return accuracy


def train_forward_forward_mnist(model, X_train, y_train, X_test, y_test,
                                 n_epochs=100, lr=0.01, threshold=0.1, 
                                 batch_size=100, eval_subset=500, verbose=True):
    """
    Train SOEN model with Forward-Forward on MNIST.
    
    Args:
        eval_subset: Number of training samples to use for evaluation (for speed)
    """
    model.train()
    
    # Layer-wise optimizers
    hidden_layer_indices = [i for i, l in enumerate(model.layers) if l.layer_type != 'Input']
    layer_optimizers = []
    for conn_key in model.connections.keys():
        conn_params = [model.connections[conn_key]]
        layer_optimizers.append(torch.optim.Adam(conn_params, lr=lr))
    
    history = {
        'loss': [],
        'train_acc': [],
        'test_acc': [],
        'goodness_pos': [],
        'goodness_neg': [],
    }
    
    N = X_train.shape[0]
    n_batches = (N + batch_size - 1) // batch_size
    
    # Subset for fast evaluation during training
    eval_idx = torch.randperm(N)[:eval_subset]
    X_train_eval = X_train[eval_idx]
    y_train_eval = y_train[eval_idx]
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_g_pos = []
        epoch_g_neg = []
        
        # Shuffle data
        perm = torch.randperm(N)
        X_shuffled = X_train[perm]
        y_shuffled = y_train[perm]
        
        for batch_idx in range(n_batches):
            start = batch_idx * batch_size
            end = min(start + batch_size, N)
            
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # Create pos/neg pairs
            X_pos, X_neg = create_positive_negative_pairs(X_batch, y_batch)
            
            # Expand to sequence
            X_pos_seq = X_pos.unsqueeze(1).expand(-1, SEQ_LEN, -1).clone()
            X_neg_seq = X_neg.unsqueeze(1).expand(-1, SEQ_LEN, -1).clone()
            
            # Forward pass
            _, layer_states_pos = model(X_pos_seq)
            _, layer_states_neg = model(X_neg_seq)
            
            # Train each layer
            batch_loss = 0
            for layer_idx, opt in zip(hidden_layer_indices, layer_optimizers):
                opt.zero_grad()
                
                act_pos = layer_states_pos[layer_idx][:, -1, :]
                act_neg = layer_states_neg[layer_idx][:, -1, :]
                
                g_pos = compute_goodness(act_pos)
                g_neg = compute_goodness(act_neg)
                
                epoch_g_pos.append(g_pos.mean().item())
                epoch_g_neg.append(g_neg.mean().item())
                
                layer_loss = forward_forward_loss(g_pos, g_neg, threshold)
                batch_loss += layer_loss.item()
                
                layer_loss.backward(retain_graph=True)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()
            
            epoch_loss += batch_loss
        
        # Fast evaluation on subset
        train_acc = evaluate_ff_mnist_fast(model, X_train_eval, y_train_eval)
        test_acc = evaluate_ff_mnist_fast(model, X_test, y_test)
        
        history['loss'].append(epoch_loss / n_batches)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['goodness_pos'].append(np.mean(epoch_g_pos))
        history['goodness_neg'].append(np.mean(epoch_g_neg))
        
        if verbose and (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: Loss={epoch_loss/n_batches:.4f}, "
                  f"Train={train_acc:.4f}, Test={test_acc:.4f}, "
                  f"G_pos={np.mean(epoch_g_pos):.4f}, G_neg={np.mean(epoch_g_neg):.4f}")
    
    return history


def evaluate_ff_mnist(model, X, y, batch_size=200):
    """
    Full evaluation (for final results, not during training).
    """
    return evaluate_ff_mnist_fast(model, X, y, batch_size, seq_len=SEQ_LEN)

## 5. Train the Model

In [ ]:
# Build model with only 12 hidden neurons
HIDDEN_DIMS = [12]
THRESHOLD = 0.1
N_EPOCHS = 100
LR = 0.01
BATCH_SIZE = 100

print(f"Training Forward-Forward MNIST classifier...")
print(f"Architecture: 794 → {HIDDEN_DIMS} → goodness")
print(f"Only {HIDDEN_DIMS[0]} hidden neurons for 10-class MNIST!")
print(f"Threshold: {THRESHOLD}")
print(f"Training samples: {N_TRAIN}, Test samples: {N_TEST}")
print("=" * 60)

torch.manual_seed(42)
model = build_ff_mnist_model(HIDDEN_DIMS)

history = train_forward_forward_mnist(
    model, X_train, y_train, X_test, y_test,
    n_epochs=N_EPOCHS, lr=LR, threshold=THRESHOLD,
    batch_size=BATCH_SIZE, verbose=True
)

print("=" * 60)
print(f"Final train accuracy: {history['train_acc'][-1]:.4f}")
print(f"Final test accuracy: {history['test_acc'][-1]:.4f}")

## 6. Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Loss
ax1 = axes[0, 0]
ax1.plot(history['loss'], color='steelblue', lw=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Forward-Forward Loss')
ax1.set_title('Training Loss')
ax1.grid(True, alpha=0.3)

# Goodness
ax2 = axes[0, 1]
ax2.plot(history['goodness_pos'], label='Positive', color='green', lw=2)
ax2.plot(history['goodness_neg'], label='Negative', color='red', lw=2)
ax2.axhline(y=THRESHOLD, color='black', linestyle='--', label=f'Threshold={THRESHOLD}')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Mean Goodness')
ax2.set_title('Goodness Separation')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Accuracy
ax3 = axes[1, 0]
ax3.plot(history['train_acc'], label='Train', color='coral', lw=2)
ax3.plot(history['test_acc'], label='Test', color='steelblue', lw=2)
ax3.axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='Random (10%)')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Accuracy')
ax3.set_title('Classification Accuracy')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 1.0)

# Separation
ax4 = axes[1, 1]
separation = [p - n for p, n in zip(history['goodness_pos'], history['goodness_neg'])]
ax4.plot(separation, color='purple', lw=2)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax4.set_xlabel('Epoch')
ax4.set_ylabel('G_pos - G_neg')
ax4.set_title('Goodness Separation')
ax4.grid(True, alpha=0.3)

plt.suptitle(f'Forward-Forward MNIST (12 hidden neurons)', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Confusion Matrix

In [ ]:
def get_predictions_ff(model, X, batch_size=200):
    """Get predictions and goodness for all samples (optimized batched version)."""
    model.eval()
    N = X.shape[0]
    all_predictions = []
    all_goodness = []
    
    with torch.no_grad():
        for start in range(0, N, batch_size):
            end = min(start + batch_size, N)
            X_batch = X[start:end]
            B = X_batch.shape[0]
            
            # Batch all 10 class hypotheses together
            X_repeated = X_batch.unsqueeze(1).expand(-1, N_CLASSES, -1).reshape(B * N_CLASSES, 784)
            y_hypotheses = torch.arange(N_CLASSES).unsqueeze(0).expand(B, -1).reshape(B * N_CLASSES)
            
            X_embedded = embed_label(X_repeated, y_hypotheses)
            X_seq = X_embedded.unsqueeze(1).expand(-1, SEQ_LEN, -1)
            
            _, layer_states = model(X_seq)
            
            total_goodness = torch.zeros(B * N_CLASSES)
            for layer_idx in range(1, len(model.layers)):
                act = layer_states[layer_idx][:, -1, :]
                total_goodness += compute_goodness(act)
            
            goodness_matrix = total_goodness.reshape(B, N_CLASSES)
            all_goodness.append(goodness_matrix)
            all_predictions.append(goodness_matrix.argmax(dim=1))
    
    return torch.cat(all_predictions), torch.cat(all_goodness)


def compute_confusion_matrix(y_true, y_pred, n_classes=10):
    """Compute confusion matrix without sklearn."""
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    for true, pred in zip(y_true, y_pred):
        cm[true, pred] += 1
    return cm


# Get test predictions
test_preds, test_goodness = get_predictions_ff(model, X_test)

# Confusion matrix (no sklearn needed)
cm = compute_confusion_matrix(y_test.numpy(), test_preds.numpy())

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title(f'Confusion Matrix (Test Acc: {history["test_acc"][-1]:.2%})')

# Add text annotations
for i in range(10):
    for j in range(10):
        text = ax.text(j, i, cm[i, j], ha='center', va='center',
                       color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.colorbar(im)
plt.tight_layout()
plt.show()

# Per-class accuracy
print("\nPer-class accuracy:")
for digit in range(10):
    mask = y_test == digit
    digit_acc = (test_preds[mask] == digit).float().mean().item()
    print(f"  Digit {digit}: {digit_acc:.2%}")

## 8. Visualize Predictions

In [ ]:
# Show some predictions
n_show = 20
fig, axes = plt.subplots(4, 5, figsize=(15, 12))

for i, ax in enumerate(axes.flat):
    if i >= n_show:
        break
    
    img = X_test[i].view(28, 28).numpy()
    true_label = y_test[i].item()
    pred_label = test_preds[i].item()
    goodness_vals = test_goodness[i].numpy()
    
    ax.imshow(img, cmap='gray')
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f'True: {true_label}, Pred: {pred_label}', color=color)
    ax.axis('off')

plt.suptitle('Forward-Forward MNIST Predictions (green=correct, red=wrong)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Show goodness distribution for a few samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    goodness_vals = test_goodness[i].numpy()
    true_label = y_test[i].item()
    pred_label = test_preds[i].item()
    
    colors = ['green' if d == true_label else 'lightgray' for d in range(10)]
    colors[pred_label] = 'red' if pred_label != true_label else 'green'
    
    ax.bar(range(10), goodness_vals, color=colors)
    ax.set_xticks(range(10))
    ax.set_xlabel('Digit')
    ax.set_ylabel('Goodness')
    status = '✓' if pred_label == true_label else '✗'
    ax.set_title(f'True: {true_label}, Pred: {pred_label} {status}')

plt.suptitle('Goodness Distribution per Digit Hypothesis', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Compare with Different Hidden Sizes

In [ ]:
# Compare different hidden layer sizes
hidden_configs = [
    [8],
    [12],
    [16],
    [24],
    [12, 12],
]

comparison_results = []

print("Comparing different architectures...")
print("=" * 60)

for hidden_dims in hidden_configs:
    torch.manual_seed(42)
    model = build_ff_mnist_model(hidden_dims)
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    history = train_forward_forward_mnist(
        model, X_train, y_train, X_test, y_test,
        n_epochs=50, lr=0.01, threshold=0.1,
        batch_size=100, verbose=False
    )
    
    comparison_results.append({
        'hidden_dims': str(hidden_dims),
        'n_params': n_params,
        'train_acc': history['train_acc'][-1],
        'test_acc': history['test_acc'][-1],
    })
    
    print(f"Hidden={str(hidden_dims):12s} | Params={n_params:6d} | "
          f"Train={history['train_acc'][-1]:.4f} | Test={history['test_acc'][-1]:.4f}")

print("=" * 60)

## 10. Conclusions

In [ ]:
print("=" * 70)
print("CONCLUSIONS: FORWARD-FORWARD MNIST WITH 12 HIDDEN NEURONS")
print("=" * 70)

print(f"\n1. ARCHITECTURE:")
print(f"   Input: 784 pixels + 10 label = 794")
print(f"   Hidden: {HIDDEN_DIMS[0]} SingleDendrite neurons")
print(f"   Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

print(f"\n2. PERFORMANCE:")
print(f"   Train accuracy: {history['train_acc'][-1]:.2%}")
print(f"   Test accuracy:  {history['test_acc'][-1]:.2%}")
print(f"   Random baseline: 10%")

print(f"\n3. HARDWARE COMPATIBILITY:")
print(f"   ✓ Goodness = mean(I²) = power measurement")
print(f"   ✓ Label embedding = optical input modulation")
print(f"   ✓ No backward pass for inference")
print(f"   ✓ Only {HIDDEN_DIMS[0]} physical neurons needed!")

print(f"\n4. INFERENCE COST:")
print(f"   10 forward passes per sample (one per digit hypothesis)")
print(f"   Compare goodness across hypotheses to classify")

print(f"\n5. SCALING:")
print(f"   More hidden neurons → better accuracy")
print(f"   But even 12 neurons achieves meaningful classification!")

print("\n" + "=" * 70)